# DS ML Project ⟡ Flight Delay Prediction Challenge

## Synopsis

**TODO** Write this paragraph

## Requirements

In [ ]:
# Data Science
import pandas as pd
import missingno as msno

# Scientific Computation
import numpy as np
import scipy.stats as stats

# Scikit-Learn Tools
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsRegressor

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

# Power predictive score
import ppscore as pps

import warnings

# Configs

Next, let us specify all required configurations to have them in one place.

In [ ]:
# Level of the warnings module
WARNINGS_LEVEL = "ignore"
warnings.filterwarnings(WARNINGS_LEVEL)

# Path to train data
PATH_DATA_TRAIN = "./data/train.csv"

# Random seed
RSEED = 42

# Resolution when storing plots in files
DPI = 600

# Matplotlib style
PLT_STYLE = "seaborn"
try:
    plt.style.use(PLT_STYLE)
except:
    warnings.warn(f"Could not load matplotlib style {PLT_STYLE}", UserWarning)

# Whether to run long computations
RUN_LONG_COMPUTATIONS = True

We read the data from file into a pandas data frame and create a copy that will incorporate our manipulations.

In [ ]:
df_0 = pd.read_csv(PATH_DATA_TRAIN)

df = df_0.copy()

In [ ]:
# print(df.head())
# print(df.shape)
# print(df.isnull().sum())
# print(df.dtypes)

df.columns

Variable definitions (according to <https://zindi.africa/competitions/flight-delay-prediction-challenge/data>):

Present in the data:

| Column      | Description                  |
|-------------|------------------------------|
| ID          | Unique identifier for the flight. |
| DATOP       | Date of flight.                 |
| FLTID       | Flight number.                  |
| DEPSTN      | Departure point (station/airport). |
| ARRSTN      | Arrival point (station/airport). |
| STD         | Scheduled Time of Departure.    |
| STA         | Scheduled Time of Arrival.      |
| STATUS      | Flight status (e.g., delayed, canceled). |
| AC          | Aircraft code.                 |
| target      | Flight delay (in minutes).      |


Not present in the data:

| Column | Description        |
|-------------|--------------------|
| ETD         | Expected Time departure |
| ETA         | Expected Time arrival   |
| ATD         | Actual Time of Departure |
| ATA         | Actual Time of arrival   |
| DELAY1      | Delay code 1            |
| DUR1        | Delay time 1            |
| DELAY2      | Delay code 2            |
| DUR2        | Delay time 2            |
| DELAY3      | Delay code 3            |
| DUR3        | Delay time 3            |
| DELAY4      | Delay code 4            |
| DUR4        | Delay time 4            |

In [ ]:
# Sorry for "statuses" ...
statuses = df["STATUS"].unique()

print("All Statuses:")
for status in statuses:
    print(f"  Number of entries of {status}: {df[df['STATUS'] == status].shape[0]}")
    print(f"  Mean: {df[df['STATUS'] == status]['target'].mean()}")
    print(f"  Median: {df[df['STATUS'] == status]['target'].median()}")

for status in statuses:
    print(status)
    df[df["STATUS"] == status]["target"].hist(
        bins=50,
        log=False,
    )
    plt.show()

| Code | Name              | Description                                                                 |
|------|-------------------|-----------------------------------------------------------------------------|
| ATA  | Actual Time Arrival| Flights that successfully landed at their destination.                     |
| DEP  | Departed          | Flights that departed but may not have completed their journey.             |
| RTR  | Returned          | Flights that took off but returned to the departure airport due to issues.  |
| SCH  | Scheduled         | Flights listed in the schedule, no delay data applicable.                   |
| DEL  | Canceled          | Flights that were canceled, treated as permanent delays.                    |

The interpretation of DEP remains a bit obscure ... in a first approximation, we drop it. Further, it is hard to measure the delay of a DEL flight (a possibility for regular flights would be to take the duration between the DEL flight and the next flight that indeed arrives plus the delay of that flight). But we also decide us simply for dropping.

In [ ]:
df = df[~df["STATUS"].isin(["DEP", "DEL"])]

In [ ]:
df["target"].hist(
    bins=100, 
    log=False,
)
plt.xlim(1, 500)
plt.show()

In [ ]:
df.head()

In [ ]:
df["DATOP"] = pd.to_datetime(df["DATOP"], format="%Y-%m-%d")
df["STD"] = pd.to_datetime(df["STD"], format="%Y-%m-%d %H:%M:%S")
df["STA"] = pd.to_datetime(df["STA"], format="%Y-%m-%d %H.%M.%S")

In [ ]:
# extract year, month, dayofweek and hour information out of column publish_time and build new column for each
df["DATOP_year"] = df["DATOP"].dt.year
df["DATOP_month"] = df["DATOP"].dt.month
df["DATOP_day"] = df["DATOP"].dt.dayofweek + 1
# df["publish_hour"]=df["publish_time"].dt.hour

In [ ]:
df["flight_time"] = (df["STA"] - df["STD"]).dt.total_seconds() / 60

In [ ]:
df.head()


In [ ]:
correlation_matrix = df.corr()

# Plot the correlation matrix as a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="Reds", fmt=".2f")
plt.title("Correlation Matrix")
plt.show()

A PP-score matrix:

In [ ]:
if RUN_LONG_COMPUTATIONS:

    cols = [
        col for col in df.columns
        if not col.startswith("DATOP_") and col not in ["ID"]
    ]
    
    df_tmp = df[cols]

    pp_scores = pps.matrix(df_tmp)[["x", "y", "ppscore"]].pivot(
        columns="x", index="y", values="ppscore"
    )

    pp_scores = pp_scores.round(2)

    plt.figure(figsize=(12, 8))

    sns.heatmap(
        pp_scores,
        vmin=0,
        vmax=1,
        cmap="Reds",
        linewidths=0.5,
        annot=True,
    )

    plt.savefig("./img/pp-scores.png", dpi=DPI, bbox_inches="tight")

    plt.plot()

Scatterplot

In [ ]:
if RUN_LONG_COMPUTATIONS:
    sns.pairplot(df)

    plt.savefig("./img/pairplot.png", dpi=DPI, bbox_inches="tight")

    plt.show()

In [ ]:
DATOP_years = df["DATOP_year"].unique()

for status in DATOP_years:
    plt.figure(figsize=(8, 4))
    df_year = df[df["DATOP_year"] == status]
    df_year["DATOP_month"].hist(bins=12)
    plt.title(f"Flight Distribution per Month – {status}")
    plt.xlabel("Month")
    plt.ylabel("Number of Flights")
    plt.xticks(range(1, 13))
    plt.tight_layout()
    plt.show()

In [ ]:
for status in DATOP_years:
    plt.figure(figsize=(8, 4))
    df_year = df[df["DATOP_year"] == status]
    df_year.groupby("DATOP_month")["target"].sum().plot(
        kind="line",
        title=f"Monthly Sum of Target for {status}",
        xlabel="Month",
        ylabel="Sum of Target",
    )

The missing months can be found in the test data set (sic!).

# Baseline Model (a.k.a. A Feeble Try), Version I

**Hypothesis**: The flight delay can be predicted from the Aircraft Code.

In [ ]:
df_encoded = pd.get_dummies(df, columns=["AC"], prefix="AC")

y = df.target
X = df_encoded.drop("target", axis=1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=RSEED
)

Fit -- Predict -- Evaluate

In [ ]:
cols = [col for col in df_encoded.columns if col.startswith("AC_")]

X_0 = X_train[cols]
y_0 = y_train
X_1 = X_test[cols]
y_1 = y_test

# model = KNeighborsRegressor(n_neighbors=3)
# # model = LinearRegression()

# model.fit(X_0, y_0)

# y_pred = model.predict(X_1)

# mse = mean_squared_error(y_1, y_pred)
# rmse = np.sqrt(mse)
# r2 = r2_score(y_1, y_pred)

# # print("Coefficients:", linreg.coef_)
# # print("Intercept:", model.intercept_)
# print("Root Mean Squared Error:", rmse)
# # print("R-squared Score:", r2)

In [ ]:
# X_test.to_csv("data/X_test.csv")
# y_test.to_csv("data/y_test.csv")

## Some Considerations About Flight Delay

In [ ]:
# die Verteilung der Verspätungen unter Abluegen und Ankuenften
# Durchschnittliche Verspätung pro Abflughafen
dep_delay = df.groupby("DEPSTN")["target"].mean().sort_values(ascending=False)

plt.figure(figsize=(12, 6))
dep_delay.plot(kind="bar", color="skyblue")
plt.title("Durchschnittliche Verspätung pro Abflughafen")
plt.xlabel("Abflughafen")
plt.ylabel("Durchschnittliche Verspätung (Minuten)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Durchschnittliche Verspätung pro Ankunftsflughafen
arr_delay = df.groupby("ARRSTN")["target"].mean().sort_values(ascending=False)

plt.figure(figsize=(12, 6))
arr_delay.plot(kind="bar", color="salmon")
plt.title("Durchschnittliche Verspätung pro Ankunftsflughafen")
plt.xlabel("Ankunftsflughafen")
plt.ylabel("Durchschnittliche Verspätung (Minuten)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Pivot-Tabelle für Heatmap
route_delay = df.pivot_table(
    index="DEPSTN", columns="ARRSTN", values="target", aggfunc="mean"
)

plt.figure(figsize=(12, 8))
sns.heatmap(route_delay, cmap="Reds", linewidths=0.5, annot=False)
plt.title("Durchschnittliche Verspätung zwischen Abflug- und Ankunftsflughäfen")
plt.xlabel("Ankunftsflughafen")
plt.ylabel("Abflughafen")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 6))
sns.boxplot(x="DEPSTN", y="target", data=df)
plt.title("Verteilung der Verspätungen pro Abflughafen")
plt.xticks(rotation=45)
plt.show()

In [ ]:
pivot = df.pivot_table(
    index="DEPSTN", columns="ARRSTN", values="target", aggfunc="mean"
)
plt.figure(figsize=(12, 8))
sns.heatmap(pivot, annot=False, cmap="Reds")
plt.title("Durchschnittliche Verspätung je Flugroute (in Minuten)")
plt.xlabel("Ankunftsflughafen")
plt.ylabel("Abflughafen")
plt.show()

In [ ]:
top_dep = df["DEPSTN"].value_counts().head(10)
avg_delay_dep = df.groupby("DEPSTN")["target"].mean().loc[top_dep.index]

summary = pd.DataFrame({"Fluganzahl": top_dep, "Ø Verspätung (Min.)": avg_delay_dep})
print(summary)

In [ ]:
df[df["DEPSTN"] == "TUN"]["target"].hist(bins=30)
plt.title("Verspätungsverteilung – Abflughafen TUN")
plt.xlabel("Verspätung in Minuten")
plt.ylabel("Anzahl Flüge")
plt.show()